# LEDVR的示例代码

In [10]:
!pip install openai langchain tiktoken faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 3.8 MB/s eta 0:00:00


In [2]:
# 设置OpenAI KEY环境变量
import os
import getpass
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


首先，我们使用文档加载器L，创建一个WebBaseLoader实例，用于从网络加载数据。在这个例子中，我们加载的是一个博客文章。加载器读取该网址的内容，并将其转换为一份文档数据。

In [3]:
from langchain.document_loaders import WebBaseLoader
loader = WebBaseLoader("http://developers.mini1.cn/wiki/luawh.html")
data = loader.load()



随后，我们使用文本嵌入模型E，将这些分割后的文本数据转换为向量数据。我们创建一个OpenAIEmbeddings实例，用于将文本转换为向量。

In [4]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


接下来，文档转换器D，我们需要将这份文档数据进行分割，转换为 Document 数据。这是为了让数据更好地适应数据连接模块的工作流程。我们创建一个RecursiveCharacterTextSplitter实例，并指定每个片段的大小为500个字符。然后，我们使用这个分割器将文档数据分割成多个片段。

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data)



我们看一下切割为多少块了：

In [6]:
len(splits)

79

然后，向量存储器V，我们创建一个向量存储器：FAISS实例，用于存储这些向量数据。

In [11]:
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(documents=splits, embedding=embedding)
vectordb

最后，检索器R，我们需要在这些数据中进行检索。我们创建一个ChatOpenAI实例和一个MultiQueryRetriever实例，用于执行检索操作。在这个例子中，我们检索的问题是“LUA的宿主语言是什么?”。

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever

question = "LUA的宿主语言是什么?"
llm = ChatOpenAI(openai_api_key="")
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vectordb.as_retriever(), llm=llm
)


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [13]:
docs = retriever_from_llm.get_relevant_documents(question)
docs[0]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Document(page_content='Lua提供了非常易于使用的扩展接口和机制：由宿主语言(通常是C或C++)提供这些功能，Lua可以使用它们，就像是本来就内置的功能一样。 其它特性:', metadata={'source': 'http://developers.mini1.cn/wiki/luawh.html', 'title': '什么是Lua编程 | 开发者脚本帮助文档', 'description': '迷你世界开发者接口文档', 'language': 'zh-CN'})